# Calculating Program Invariants 

We wanted to demonstrate how grobner basis and the ideal-variety correspondence from 
computational algebraic geometry can be used to reason about programs. 

Roughly based on the works by 
 - Sankaranarayanan, Sipma and Manna, POPL 2004.
 - Carbonell and Kapur, ISSAC 2004, SAS 2004.
   


In [59]:
import sympy as sp 

Let's start with a simple while loop: 

Initially: x = 0, y = 0

while ( ... ){ x, y = x + 1, y + 2* x + 1 } 

We want to compute a polynomial loop invariant involving $x, y$.

In [60]:
# Let us declare program variables and somehow create a program structure.

prog_vars = [sp.Symbol('x'), sp.Symbol('y')]
primed_prog_vars = [sp.Symbol('xp'), sp.Symbol('yp')]

[x, y] = prog_vars
[xp, yp] = primed_prog_vars

In [61]:
init_cond = [x, y] # this represents the constraints x = 0, y = 0

trans_rel = [xp - x -1, yp - y - 2*x - 1]

loop_transition_relations = [trans_rel]

In [62]:
def mk_groebner_basis(constrs, prog_vars):
    return sp.groebner(constrs, prog_vars, order='grevlex')

In [63]:
def compute_union_through_product(Gi, Gj, prog_vars):
    # multiply all polynomials in I and J together
    I = Gi.exprs
    J = Gj.exprs
    # Check if one of them is contained in the other. 
    if check_containment_by_reduction(Gi, Gj):
        # var(G1) \substeq var(G0)
        return Gi
    if check_containment_by_reduction(Gj, Gi):
        return Gj
    
    J = [p for p in J if (Gi.reduce(p)[1] != 0)]
    IJ = [ p * q for p in I for q in J ] 
    grb_IJ = sp.groebner(IJ, prog_vars, order='grevlex')
    return grb_IJ

In [64]:
def compute_union_through_elimination(Gi, Gj, prog_vars):
    tmp = sp.Symbol('tmp__') # create a new symbol tmp
    I = Gi.exprs
    J = Gj.exprs 
    IJ = [tmp*p for p in I] + [(1-tmp)*q for q in J]
    grb_IJ = sp.groebner(IJ, [tmp]+prog_vars, order='lex')
    new_bases = [ p for p in grb_IJ.exprs if not(p.has_free(tmp)) ]
    return sp.groebner(new_bases, prog_vars, order='grevlex')

In [65]:
def compute_union(Gi, Gj, prog_vars):
    return compute_union_through_elimination(Gi, Gj, prog_vars)

In [66]:
def compute_post_cond_elim( I0, trs_rel, prog_vars, primed_prog_vars):
    combined =  I0.exprs + trs_rel 
    # use lexicographic grobner bases
    combined_grb = sp.groebner(combined, prog_vars + primed_prog_vars, order='lex')
    def check_if_poly_has_prog_vars(poly):
        return any(poly.has_free(v) for v in prog_vars)
    exprs = combined_grb.exprs
    exprs_primed = [ e for e in exprs if not check_if_poly_has_prog_vars(e) ]
    # now we need to substitute
    subst = [( xp, x) for (xp, x) in zip(primed_prog_vars, prog_vars)]
    exprs_subst = [sp.poly( e.subs(subst), prog_vars) for e in exprs_primed]
    
    return mk_groebner_basis(exprs_subst, prog_vars)

In [67]:
from functools import reduce

def compute_post_and_join(I, I0, trs_rels, prog_vars, primed_prog_vars):
    # compute all the post-conditions 
    post_conds = [compute_post_cond_elim(I, tr_rel, prog_vars, primed_prog_vars) for tr_rel in trs_rels] 
    # compute the joins with I0
    J = reduce( lambda I,J: compute_union(I, J, prog_vars), post_conds, I0)
    return J
    

In [68]:
I0 = mk_groebner_basis(init_cond, prog_vars)
J1 = compute_post_cond_elim(I0, trans_rel, prog_vars, primed_prog_vars)
print(J1)

GroebnerBasis([x - 1, y - 1], x, y, domain='ZZ', order='grevlex')


In [69]:

I1 = compute_post_and_join(I0, I0, loop_transition_relations, prog_vars, primed_prog_vars) 
print(I1)

GroebnerBasis([y**2 - y, x - y], x, y, domain='ZZ', order='grevlex')


In [70]:
I2 = compute_post_and_join(I1, I0, loop_transition_relations, prog_vars, primed_prog_vars)
print(I2)

GroebnerBasis([x**2 - y, x*y + 2*x - 3*y, y**2 + 6*x - 7*y], x, y, domain='ZZ', order='grevlex')


In [71]:
I3 = compute_post_and_join(I2, I0, loop_transition_relations, prog_vars, primed_prog_vars)
print(I3)

GroebnerBasis([y**3 - 15*y**2 - 60*x + 74*y, x**2 - y, 6*x*y - y**2 + 6*x - 11*y], x, y, domain='ZZ', order='grevlex')


In [72]:
I4 = compute_post_and_join(I3, I0, loop_transition_relations, prog_vars, primed_prog_vars)
print(I4)

GroebnerBasis([x*y**2 + 35*x*y - 10*y**2 + 24*x - 50*y, y**3 + 300*x*y - 65*y**2 + 240*x - 476*y, x**2 - y], x, y, domain='ZZ', order='grevlex')


## Widening Operator 

There are many ways to define a widening operator. A simple widening operator is the analog of standard widening. 

 - $I_n \nabla I_{n+1}$ where $I_{n+1} \subseteq I_n$.
 - Note the ordering of ideals is flipped from what one would usually consider for abstract interpretation.
 - This is because the abstract domain we would like to work with is that of algebraic varieties:
    - Ideals are a representation of varieties. 


``Standard Widening'' $I_n \nabla I_{n+1}$. Let $I_n = \langle g_1, \ldots, g_k \rangle $. 
  - Choose $g_i \in \mathsf{generators}(I_n)$ such that $g_i \in I_{n+1}$.

Note that using standard widening guarantees convergence. 

In [73]:
def widen(G0, G1, prog_vars):
    result = [] 
    for p in G0.exprs: 
        (_, res) = G1.reduce(p)
        if res == 0:
            result.append(p)
    Gp = sp.groebner(result, prog_vars, order='grevlex')
    return Gp

In [74]:
def check_containment_by_reduction(G0, G1):
    # use a sufficient condition to check if var(G1) \subseteq var(G0)
    # in other words ideal(G0) \subseteq ideal(G1)
    # therefore, every generator of G0 is reduced by G1 to 0
    I0 = G0.exprs
    for p in I0:
        res = G1.reduce(p)[1]
        if res != 0:
            return False
    return True
        
    

In [75]:
def perform_abstract_interpretation(init_cond, trs_relations, prog_vars, primed_prog_vars, widen_delay=4):
    count = 1
    I0 = mk_groebner_basis(init_cond, prog_vars)
    I = I0
    terminated = False
    while not(terminated):
        print(f'Interation # {count}')
        J = compute_post_and_join(I, I0, trs_relations, prog_vars, primed_prog_vars)
        print(f'\t J= {J}')
        if (count < widen_delay):
            I = J
        else: 
            print('\t Widening')
            n_polys = len(I.exprs)
            I_hat = widen(I, J, prog_vars)
            print(f'\t After Widening: {I_hat}')
            if check_containment_by_reduction(I, I_hat):
                return I_hat
            I = I_hat
        count = count + 1 
    return I

In [76]:
I = perform_abstract_interpretation(init_cond, loop_transition_relations, prog_vars, primed_prog_vars)
print(I)

Interation # 1
	 J= GroebnerBasis([y**2 - y, x - y], x, y, domain='ZZ', order='grevlex')
Interation # 2
	 J= GroebnerBasis([x**2 - y, x*y + 2*x - 3*y, y**2 + 6*x - 7*y], x, y, domain='ZZ', order='grevlex')
Interation # 3
	 J= GroebnerBasis([y**3 - 15*y**2 - 60*x + 74*y, x**2 - y, 6*x*y - y**2 + 6*x - 11*y], x, y, domain='ZZ', order='grevlex')
Interation # 4
	 J= GroebnerBasis([x*y**2 + 35*x*y - 10*y**2 + 24*x - 50*y, y**3 + 300*x*y - 65*y**2 + 240*x - 476*y, x**2 - y], x, y, domain='ZZ', order='grevlex')
	 Widening
	 After Widening: GroebnerBasis([x**2 - y], x, y, domain='ZZ', order='grevlex')
Interation # 5
	 J= GroebnerBasis([x**2 - y], x, y, domain='ZZ', order='grevlex')
	 Widening
	 After Widening: GroebnerBasis([x**2 - y], x, y, domain='ZZ', order='grevlex')
GroebnerBasis([x**2 - y], x, y, domain='ZZ', order='grevlex')


## Integer Division (Manna'1974)

```{.python} 
# Integer division from Manna 1974
def div(x1, x2):
  (y1, y2, y3, y4) = (x1, x2, 1, 0)
  while y1 >= y2: # Inv: y2 - y3 * x2 == 0, y1*y3 + y2*y4 − y3*x1 == 0
     (y2, y3) = (2 * y2, 2 * y3)
  while True: # Inv: y2 - y3 * x2 == 0, y1*y3 + y2*y4 − y3*x1 == 0
     if y1 >= y2: 
        (y1, y4) = (y1 - y2, y3 + y4)
     if y3 == 1: 
        return y4, y1 # Prove: x1 = y1 + x2 * y4
     (y2, y3) = (y2 / 2, y3 / 2) # y2, y3 are  even, before division
```

In [77]:
prog_vars_names= ['x1', 'x2', 'y1', 'y2', 'y3', 'y4']
prog_vars_primed_names = ['x1p', 'x2p', 'y1p', 'y2p', 'y3p', 'y4p']
prog_vars = [sp.Symbol(p) for p in prog_vars_names]
prog_vars_primed = [sp.Symbol(p) for p in prog_vars_primed_names]
[x1, x2, y1, y2, y3, y4] = prog_vars 
[x1p, x2p, y1p, y2p, y3p, y4p] = prog_vars_primed 


In [78]:
init_cond = [x1 - y1, x2 - y2, y3 - 1, y4 ]
I0 = mk_groebner_basis(init_cond, prog_vars)




In [79]:
trans_rels_loop_0 = [ [x1p - x1, x2p - x2, y2p - sp.Rational(2,1) * y2, y3p -  sp.Rational(2,1) * y3, y1p - y1 , y4p - y4] ]

print(compute_post_cond_elim(I0, trans_rels_loop_0[0], prog_vars, prog_vars_primed))

GroebnerBasis([x1 - y1, 2*x2 - y2, y3 - 2, y4], x1, x2, y1, y2, y3, y4, domain='ZZ', order='grevlex')


In [80]:
I = perform_abstract_interpretation(init_cond, trans_rels_loop_0, prog_vars, prog_vars_primed)

Interation # 1
	 J= GroebnerBasis([2*x2**2 - 3*x2*y2 + y2**2, x2*y3 - y2, y2*y3 + 2*x2 - 3*y2, y3**2 - 3*y3 + 2, x1 - y1, y4], x1, x2, y1, y2, y3, y4, domain='ZZ', order='grevlex')
Interation # 2
	 J= GroebnerBasis([8*x2**3 - 14*x2**2*y2 + 7*x2*y2**2 - y2**3, y2**2*y3 - 8*x2**2 + 14*x2*y2 - 7*y2**2, y2*y3**2 - 7*y2*y3 - 8*x2 + 14*y2, y3**3 - 7*y3**2 + 14*y3 - 8, x2*y3 - y2, x1 - y1, y4], x1, x2, y1, y2, y3, y4, domain='ZZ', order='grevlex')
Interation # 3
	 J= GroebnerBasis([64*x2**4 - 120*x2**3*y2 + 70*x2**2*y2**2 - 15*x2*y2**3 + y2**4, y2**3*y3 + 64*x2**3 - 120*x2**2*y2 + 70*x2*y2**2 - 15*y2**3, y2**2*y3**2 - 15*y2**2*y3 + 64*x2**2 - 120*x2*y2 + 70*y2**2, y2*y3**3 - 15*y2*y3**2 + 70*y2*y3 + 64*x2 - 120*y2, y3**4 - 15*y3**3 + 70*y3**2 - 120*y3 + 64, x2*y3 - y2, x1 - y1, y4], x1, x2, y1, y2, y3, y4, domain='ZZ', order='grevlex')
Interation # 4
	 J= GroebnerBasis([1024*x2**5 - 1984*x2**4*y2 + 1240*x2**3*y2**2 - 310*x2**2*y2**3 + 31*x2*y2**4 - y2**5, y2**4*y3 - 1024*x2**4 + 1984*x2**3*y2

In [81]:
trans_rels_loop_1 = [ [ 2* y2p - y2, 2*y3p  - y3, y1p - y1, y4p - y4, x2p - x2, x1p - x1 ], # y2 = y2/2, y3 = y3/2
                      [ y1p - y1 + y2, 2*y3p - y3, 2* y2p - y2, y4p - y4 -y3, x2p - x2, x1p - x1]
                    ]

In [82]:
I_1 = perform_abstract_interpretation(I.exprs, trans_rels_loop_1, prog_vars, prog_vars_primed)

Interation # 1
	 J= GroebnerBasis([x1**2 - 2*x1*y1 + y1**2 - 2*x1*y2 + 2*y1*y2, x1*y3 - y1*y3 - y2*y4, x2*y3 - y2, x1*y4 - y1*y4 - 2*y2*y4, x2*y4 - x1 + y1, 2*y3*y4 - y4**2], x1, x2, y1, y2, y3, y4, domain='ZZ', order='grevlex')
Interation # 2
	 J= GroebnerBasis([x1**4 - 4*x1**3*y1 + 6*x1**2*y1**2 - 4*x1*y1**3 + y1**4 - 12*x1**3*y2 + 36*x1**2*y1*y2 - 36*x1*y1**2*y2 + 12*y1**3*y2 + 44*x1**2*y2**2 - 88*x1*y1*y2**2 + 44*y1**2*y2**2 - 48*x1*y2**3 + 48*y1*y2**3, x1**3*y4 - 3*x1**2*y1*y4 + 3*x1*y1**2*y4 - y1**3*y4 - 12*x1**2*y2*y4 + 24*x1*y1*y2*y4 - 12*y1**2*y2*y4 + 44*x1*y2**2*y4 - 44*y1*y2**2*y4 - 48*y2**3*y4, 48*y2**2*y3*y4 - x1**2*y4**2 + 2*x1*y1*y4**2 - y1**2*y4**2 + 12*x1*y2*y4**2 - 12*y1*y2*y4**2 - 44*y2**2*y4**2, 48*y2*y3**2*y4 - 44*y2*y3*y4**2 - x1*y4**3 + y1*y4**3 + 12*y2*y4**3, 48*y3**3*y4 - 44*y3**2*y4**2 + 12*y3*y4**3 - y4**4, x1*y3 - y1*y3 - y2*y4, x2*y3 - y2, x2*y4 - x1 + y1], x1, x2, y1, y2, y3, y4, domain='ZZ', order='grevlex')
Interation # 3
	 J= GroebnerBasis([x1**8 - 8*x1